In [3]:
import numpy as np

# --- 1. Imports for OLD Versions (qiskit-nature 0.7.2) ---
from qiskit_nature.drivers.second_q import PySCFDriver
from qiskit_nature.mappers.second_q import JordanWignerMapper
from qiskit_nature.converters.second_q import QubitConverter

# --- 2. Imports for OLD Versions (qiskit-algorithms 0.4.0) ---
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

# --- 3. Imports for Backend (QuantumInstance) ---
# You must 'pip install qiskit-aer' for this to work
from qiskit_aer import Aer
from qiskit.utils import QuantumInstance

# --- 1. Define the H2 Molecule (Problem) ---
print("Setting up H2 molecule (for qiskit-nature 0.7.2)...")

# In v0.7.2, the driver takes an atom string directly
driver = PySCFDriver(
    atom="H 0.0 0.0 0.0; H 0.0 0.0 0.74",
    unit="angstrom",
    charge=0,
    spin=0,
    basis="sto3g",
)

# Run the driver to get the problem
es_problem = driver.run()

# --- 2. Map to a Qubit Hamiltonian ---
print("Mapping to qubit Hamiltonian...")

mapper = JordanWignerMapper()
# The converter takes the mapper in its constructor
# 'z2symmetries=True' is the old way to ask for tapering
converter = QubitConverter(mapper, two_qubit_reduction=True, z2symmetries=True)

# Get the fermionic operator
fermionic_op = es_problem.second_q_ops()["ElectronicEnergy"]

# Convert to qubit op
qubit_op = converter.convert(
    fermionic_op,
    num_particles=es_problem.num_particles,
)

print(f"\nNumber of qubits after tapering: {qubit_op.num_qubits}")
print("Tapered Hamiltonian:")
print(qubit_op)

# --- 3. Set up QAOA Algorithm (for qiskit-algorithms 0.4.0) ---
print("\nConfiguring QAOA...")

optimizer = COBYLA(maxiter=100)

# We use QuantumInstance instead of Sampler
backend = Aer.get_backend("statevector_simulator")
quantum_instance = QuantumInstance(backend)

p_layers = 2

# Instantiate QAOA with the quantum_instance
qaoa = QAOA(optimizer=optimizer, reps=p_layers, quantum_instance=quantum_instance)

# --- 4. Run QAOA and Find Ground State ---
print(f"Running QAOA with p={p_layers} layers...")

result = qaoa.compute_minimum_eigenvalue(qubit_op)

print("\n--- Results ---")

# In this version, we must manually add back the nuclear repulsion energy
# 'result.eigenvalue' is just the electronic part
nuclear_repulsion_energy = es_problem.nuclear_repulsion_energy
qaoa_energy = result.eigenvalue + nuclear_repulsion_energy

# Get the exact FCI energy from the problem
fci_energy = es_problem.reference_energy

print(f"QAOA Ground State Energy: {qaoa_energy:.6f} Ha")
print(f"FCI (Exact) Energy:     {fci_energy:.6f} Ha")
print(f"Error:                    {np.abs(qaoa_energy - fci_energy):.6f} Ha")

ModuleNotFoundError: No module named 'qiskit_nature.drivers'

In [2]:
!pip list |grep qiskit

qiskit                    2.2.2
qiskit-algorithms         0.4.0
qiskit-ibm-runtime        0.39.0
qiskit-nature             0.7.2
qiskit-qasm3-import       0.6.0
